In [ ]:
#!pip install easydict tqdm

In [2]:
#pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" .

In [3]:
import argparse

import torch
from torchvision import utils
from model import Generator
from tqdm import tqdm
import easydict
import numpy as np
from matplotlib import pyplot as plt
import cv2


C:\Users\melon\anaconda3\lib\site-packages\torch\utils\cpp_extension.py:274: UserWarning: Error checking compiler version for cl: [WinError 2] 지정된 파일을 찾을 수 없습니다
  warnings.warn('Error checking compiler version for {}: {}'.format(compiler, error))


In [14]:
from easydict import EasyDict as edict
args = edict({'size':1024, #웬만하면 1024x1024로 고정
              'channel_multiplier':2,
              'truncation_mean': 4096,
              'truncation': 0.7, #0.7~0.8주면 artifact줄어듦(정상적인 이미지가 나올 것임)
              'sample': 1,
              'ckpt':'./stylegan2-ffhq-config-f.pt'})

In [15]:
device='cpu'
args.latent = 512
args.n_mlp = 8

g_ema = Generator(
    args.size, args.latent, args.n_mlp, channel_multiplier=args.channel_multiplier
).to(device)
checkpoint = torch.load(args.ckpt, map_location='cpu')

g_ema.load_state_dict(checkpoint["g_ema"])

if args.truncation < 1:
    with torch.no_grad():
        mean_latent = g_ema.mean_latent(args.truncation_mean)
else:
    mean_latent = None


# application

## 1. age

In [16]:
age_vector = torch.from_numpy(np.load('./stylegan2directions/age.npy'))

In [36]:
# age들어가는 과정
age = [-20,-10,0,10,20]
with torch.no_grad():
    g_ema.eval()
    for i in tqdm(range(5)):
        # generate random latent vector
#         sample_z = torch.randn(args.sample, args.latent, device=device)
        #torch.random.seed(1000)
        #torch.random.seed()
        torch.manual_seed(101)
        random_latent = torch.randn(args.sample, args.latent, device=device) #나중에는 임의로 말고 특정 latent줄 수 있음
        random_latent = g_ema.style(random_latent)
        random_latent = random_latent.unsqueeze(1).repeat(1, 18, 1) #latent사이즈 늘려줌(?)
        random_latent = random_latent + age[i] * age_vector.unsqueeze(0) #나이 들어가는 과정

        sample, _ = g_ema(
            [random_latent.to(dtype=torch.float)],
            truncation=args.truncation,
            truncation_latent=mean_latent,
            input_is_latent=True
        )

        utils.save_image(
            sample,
            f"sample/age/{str(i).zfill(6)}.png",
            nrow=1,
            normalize=True,
            range=(-1, 1),
        ) # 실행시 sample폴더에 이미지 생성됨

100%|██████████| 5/5 [00:31<00:00,  6.30s/it]


## 2. gender

In [31]:
gender_vector = torch.from_numpy(np.load('./stylegan2directions/gender.npy'))

In [42]:
# 성별 바뀌는 과정
gender = [0,4,8,12,16]
with torch.no_grad():
    g_ema.eval()
    for i in tqdm(range(5)):
        # generate random latent vector
#         sample_z = torch.randn(args.sample, args.latent, device=device)
        #torch.random.seed(1000)
        #torch.random.seed()
        torch.manual_seed(101)
        random_latent = torch.randn(args.sample, args.latent, device=device) #나중에는 임의로 말고 특정 latent줄 수 있음
        random_latent = g_ema.style(random_latent)
        random_latent = random_latent.unsqueeze(1).repeat(1, 18, 1) #latent사이즈 늘려줌(?)
        random_latent = random_latent + gender[i] * gender_vector.unsqueeze(0) #나이 들어가는 과정

        sample, _ = g_ema(
            [random_latent.to(dtype=torch.float)],
            truncation=args.truncation,
            truncation_latent=mean_latent,
            input_is_latent=True
        )

        utils.save_image(
            sample,
            f"sample/gender/{str(i).zfill(6)}.png",
            nrow=1,
            normalize=True,
            range=(-1, 1),
        ) # 실행시 sample폴더에 이미지 생성됨

100%|██████████| 5/5 [00:30<00:00,  6.16s/it]


Toonify yourself
"""
https://colab.research.google.com/drive/1s2XPNMwf6HDhrJ1FMwlW1jl-eQ2-_tlk?usp=sharing#scrollTo=cwVXBFaSuoIU
"""